In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import qc, config, tools

from importlib import reload

pd.options.display.max_rows = 9999
pd.options.display.max_columns = 9999
pd.set_option('precision', 7)

In [2]:
reload(qc)

a = qc.QueryQC()
a.test = "h"
a.scale = 1
a.cid = "01"
a.stream_n = 1
a.desc =  f"qc_query"
a.seq_id = "NA"

a.verbose = False
a.verbose_query = True
a.verbose_query_n = False
a.qual = False
a.save = True
a.verbose_iter = False

a.set_timestamp_dir()

query_n = 6

a.run_single(query_n = query_n)

#seq = tools.tpc_stream(test="ds", n=1)
#seq = [26]
#a.run(seq=seq)

df_results = a.compare()

Result Folder Name:
/home/colin/code/bq_snowflake_benchmark/results/bqsf_h_01_results--qc_query-2020-06-19_15:17:56.360693
SNOWFLAKE QUERY TEXT
ALTER SESSION SET TIMEZONE = 'UTC'

SNOWFLAKE QUERY TEXT
ALTER SESSION SET QUERY_TAG = 'h_1gb_01-xx-qc_query'

SNOWFLAKE QUERY TEXT
ALTER SESSION SET USE_CACHED_RESULT=false

SNOWFLAKE QUERY TEXT
USE WAREHOUSE TEST9000

SNOWFLAKE QUERY TEXT
USE DATABASE h_1GB_01

SNOWFLAKE QUERY TEXT
ALTER SESSION SET QUERY_TAG = 'h_1gb_01-q6-na-qc_query'

SNOWFLAKE QUERY TEXT
select
	sum(l_extendedprice * l_discount) as revenue
from
	lineitem
where
	l_shipdate >= to_date('1994-01-01')
	and l_shipdate < dateadd(year, 1, to_date('1994-01-01'))
	and l_discount between 0.04 - 0.01 and 0.04 + 0.01
	and l_quantity < 24
;



Snowflake Query Stream Done!
Total Time Elapsed: 0 days 00:00:00.785228

BIGQUERY QUERY TEXT
select
	sum(l_extendedprice * l_discount) as revenue
from
	lineitem
where
	l_shipdate >= cast('1994-01-01' as date)
	and l_shipdate < date_add(cast('1994

In [3]:
df_results

,fp_bq,q_bq,fp_sf,q_sf,equal,equal_percent
1,/home/colin/code/bq_snowflake_benchmark/result...,6,/home/colin/code/bq_snowflake_benchmark/result...,6,True,1.0


## Query Result Comparison

In [4]:
df_sf = a.result_sf
df_bq = a.result_bq

In [5]:
df_sf.head()

,revenue
0,82433974.4840


In [6]:
df_bq.head()

,revenue
0,8.2433974e+07


In [7]:
len(df_sf), len(df_bq)

(1, 1)

In [8]:
df_sf = tools.to_consistent(df_sf, n=1)
df_bq = tools.to_consistent(df_bq, n=1)

In [9]:
df_sf.dtypes

revenue    object
dtype: object

In [10]:
qc.assert_equal(df_sf, df_bq)

True

In [11]:
def color_false(val):
    mapper = {True: 'white', False: 'red'}
    return 'color: %s' % mapper[val]

In [12]:
diff = df_sf.eq(df_bq)
diff.loc[0:5].style.applymap(color_false)

,revenue
0,True


In [13]:
df_diff = diff.sum().sum() / (diff.shape[0] * diff.shape[1])
print("Percent Data Match:", df_diff)

Percent Data Match: 1.0


In [14]:
diff.sum()/diff.shape[0]

revenue    1.0
dtype: float64

In [15]:
diff.style.applymap(color_false)

,revenue
0,True
